In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt
import jax
from jax import lax
from deluca.envs import PlanarQuadrotor
from deluca.agents import ILQR


/home/alexjyu/miniconda3/envs/ealstm/lib/python3.7/site-packages/jax/lib/xla_bridge.py:123: UserWarning: No GPU/TPU found, falling back to CPU.
  warnings.warn('No GPU/TPU found, falling back to CPU.')
/home/alexjyu/miniconda3/envs/ealstm/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [3]:
agent = ILQR()
agent.train(PlanarQuadrotor(wind=0), 10)

initial cost:374122.12
(iLQR): t = 0, r = 2, c = 3861.069580078125, alpha = 1.100000023841858
(iLQR): t = 1, r = 3, c = 219.2563934326172, alpha = 1.2100000381469727
(iLQR): t = 2, r = 4, c = 69.37631225585938, alpha = 1.3310000896453857
(iLQR): t = 3, r = 5, c = 54.80947494506836, alpha = 1.4641001224517822
(iLQR): t = 4, r = 6, c = 52.38077926635742, alpha = 1.6105101108551025
(iLQR): t = 5, r = 7, c = 51.87367248535156, alpha = 1.7715611457824707
(iLQR): t = 6, r = 9, c = 51.81843566894531, alpha = 1.7715610265731812
(iLQR): t = 7, r = 12, c = 51.054931640625, alpha = 1.3309999704360962
(iLQR): t = 8, r = 13, c = 51.01693344116211, alpha = 1.4641000032424927
(iLQR): t = 9, r = 15, c = 50.95909118652344, alpha = 1.4640998840332031


(DeviceArray(50.95909, dtype=float32),
 ['(iLQR): t = 0, r = 2, c = 3861.069580078125, alpha = 1.100000023841858',
  '(iLQR): t = 1, r = 3, c = 219.2563934326172, alpha = 1.2100000381469727',
  '(iLQR): t = 2, r = 4, c = 69.37631225585938, alpha = 1.3310000896453857',
  '(iLQR): t = 3, r = 5, c = 54.80947494506836, alpha = 1.4641001224517822',
  '(iLQR): t = 4, r = 6, c = 52.38077926635742, alpha = 1.6105101108551025',
  '(iLQR): t = 5, r = 7, c = 51.87367248535156, alpha = 1.7715611457824707',
  '(iLQR): t = 6, r = 9, c = 51.81843566894531, alpha = 1.7715610265731812',
  '(iLQR): t = 7, r = 12, c = 51.054931640625, alpha = 1.3309999704360962',
  '(iLQR): t = 8, r = 13, c = 51.01693344116211, alpha = 1.4641000032424927',
  '(iLQR): t = 9, r = 15, c = 50.95909118652344, alpha = 1.4640998840332031'])

In [4]:
def loop(context, x):
    env, agent = context
    control = agent(env.state)
    _, reward, _, _ = env.step(control)
    return (env, agent), reward

In [5]:
 # for loop version
T = 100
env = PlanarQuadrotor(wind=0)
print(env.reset())
reward = 0
for i in range(T):
    (env, agent), r = loop((env, agent), 0)
    reward += r
reward_forloop = reward
print('reward_forloop = ' + str(reward_forloop))

[1. 1. 0. 0. 0. 0.]
reward_forloop = 50.95909


In [8]:
# scan version
# env = PlanarQuadrotor(placebo=0)
agent.reset()
print(env.reset())
xs = jnp.array(jnp.arange(T))
_,reward_scan = lax.scan(loop, (env, agent), xs)

# correctness test
print('reward_scan sum = ' + str(jnp.sum(reward_scan)))

[1. 1. 0. 0. 0. 0.]
reward_scan sum = 51.81845688524296
